# 01. Acquisition et Consolidation des Données

## 📝 Note Méthodologique : Objectifs et Stratégie d'Acquisition

### 0. Objectif du Projet (Définition du besoin)
Pour répondre aux exigences du devoir, nous avons besoin d'un jeu de données structuré en "Panel" (Pays x Années) permettant deux analyses distinctes :
1.  **Analyse Transversale (Cross-Section) :** Comparer les pays à une date fixe pour tester l'hypothèse de Kuznets (Relation PIB/CO2).
2.  **Analyse Temporelle (Time Series) :** Modéliser et prévoir l'évolution des émissions d'un pays spécifique (ex: USA) à travers le temps.

### 1. Tentative Initiale (API WB Banque Mondiale)
J'ai initialement tenté une extraction automatisée via `wbgapi` sur les indicateurs standards (`EN.ATM.CO2E.PC` et `NY.GDP.PCAP.PP.KD`).
*   **Résultat :** Échecs techniques répétés (Timeouts / JSON decoding errors) dus à une instabilité des serveurs de l'API.

### 2. Vérification Manuelle
Une recherche sur le portail [World Bank Data360](https://data360.worldbank.org/) a confirmé l'existence des séries, validant la pertinence économique du sujet malgré l'accessibilité technique difficile.

### 3. Solution Technique Retenue (OWID)
Pour contourner le blocage sans sacrifier la qualité, j'utilise le **repository officiel de "Our World in Data" (OWID)**.
*   **Avantage :** Ce dataset unique agrège exactement les mêmes sources (Global Carbon Project + Banque Mondiale) et offre la double profondeur historique et géographique nécessaire.

In [ ]:
import pandas as pd
import os

# =============================================================================
# ⚙️ CONFIGURATION DU PROJET — PERSONNALISABLE & REPRODUCTIBLE
# =============================================================================
RAW_DATA_DIR = os.path.join("..", "data", "01_raw")                              # Dossier de destination (respecte la structure projet)
OUTPUT_FILENAME = "kuznets_data_final.csv"                                       # Nom du fichier exporté
url = "https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv" # URL du dataset consolidé OWID

# =============================================================================
# ÉTAPE 1 : GÉNÉRATION DU FICHIER SOURCE (SOURCE : OUR WORLD IN DATA)
# =============================================================================
print("🌍 TÉLÉCHARGEMENT DU DATASET VIA OWID (CONTOURNEMENT API)...")

try:
    # Lecture du flux de données
    df_raw = pd.read_csv(url)
    
    # Sélection des colonnes pour mon projet Kuznets
    # 'co2_per_capita' = Émissions CO2 (tonnes/hab)
    # 'gdp' = PIB par habitant (ajusté inflation/PPA)
    df = df_raw[['country', 'year', 'iso_code', 'co2_per_capita', 'gdp']].copy()

    # Nettoyage (On garde seulement les pays, pas les continents)
    df = df.dropna(subset=['iso_code', 'co2_per_capita', 'gdp'])
    
    # Restriction temporelle (1990-2020) pour avoir une base cohérente et dense
    df = df[(df['year'] >= 1990) & (df['year'] <= 2020)]

    # --- SAUVEGARDE ROBUSTE ---
    
    # 1. Créer le dossier s'il n'existe pas (évite l'erreur FileNotFoundError)
    os.makedirs(RAW_DATA_DIR, exist_ok=True)
    
    # 2. Construire le chemin complet (compatible Windows/Linux/Mac)
    full_path = os.path.join(RAW_DATA_DIR, OUTPUT_FILENAME)
    
    # 3. Sauvegarde
    df.to_csv(full_path, index=False)
    
    print("✅ SUCCÈS !")
    print(f"📁 Le fichier a été sauvegardé ici : {full_path}")
    print("👉 Tu peux maintenant le retrouver dans ton dossier 'data/01_raw/'")

except Exception as e:
    print(f"❌ Erreur : {e}")

🌍 TÉLÉCHARGEMENT DU DATASET VIA OWID (CONTOURNEMENT API)...
✅ SUCCÈS !
📁 Le fichier a été sauvegardé ici : ../data/01_raw/kuznets_data_final.csv
👉 Tu peux maintenant le retrouver dans ton dossier 'data/01_raw/'


In [4]:
# Petit contrôle qualité immédiat
if os.path.exists(full_path):
    df_check = pd.read_csv(full_path)
    print("Aperçu des données :")
    display(df_check.head())
    print("\nStats descriptives :")
    display(df_check[['co2_per_capita', 'gdp']].describe().round(2))

Aperçu des données :


,country,year,iso_code,co2_per_capita,gdp
0,Afghanistan,1990,AFG,0.168,1.306598e+10
1,Afghanistan,1991,AFG,0.156,1.204736e+10
2,Afghanistan,1992,AFG,0.112,1.267754e+10
3,Afghanistan,1993,AFG,0.100,9.834582e+09
4,Afghanistan,1994,AFG,0.089,7.919856e+09



Stats descriptives :


,co2_per_capita,gdp
count,5081.00,5.081000e+03
mean,4.98,4.706262e+11
std,8.46,1.615135e+12
min,0.02,2.571720e+08
25%,0.60,1.827867e+10
50%,2.58,5.828850e+10
75%,6.85,2.814887e+11
max,364.79,2.415184e+13
